In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from sweref99 import projections
from datetime import datetime, date

In [2]:
fire_risk_path='./data/Brandriskdata 2000-2020.csv'
reported_fires_path='./data/Insatser till brand i skog och mark 2000-2020.xlsx'
#'PunktID': str, 'E': str, 'N': str,
rtypes={ 'PunktID': str}

In [3]:
#Reading csv file from path with dtypes
reported_fire_risk = pd.read_csv(fire_risk_path, sep=';', dtype=rtypes, )
reported_fires = pd.read_excel(reported_fires_path)

In [4]:
risk_sample = reported_fire_risk
fires_sample = reported_fires

In [6]:
print(risk_sample.isnull().sum(axis = 0))
print("Total rows: ")
print(risk_sample.shape[0])

PunktID                 0
E                       0
N                       0
Kommun                  2
Datum                   2
Temp                14617
Tmedel              38049
Nederbord           23520
RH                  33459
Vindhastighet        3991
Vindriktning     12310774
FFMC              1701172
DMC               1701172
DC                1701172
ISI               1701172
BUI               1701172
FWI               1701172
FWI_index         1701172
HBV_o             1404342
HBV_u             1404342
HBV               1404342
HBV_index         1404342
Gras              6855866
dtype: int64
Total rows: 
19188859


In [ ]:
risk_sample[['E','N']] = risk_sample[['E','N']].fillna(0)
risk_sample['N'] = risk_sample['N'].astype(int)
risk_sample['E'] = risk_sample['E'].astype(int)

In [7]:
risk_sample.dropna(subset=['FWI'], how='all', inplace=True)
risk_sample.dropna(subset=['Tmedel'], how='all', inplace=True)


In [8]:
print(risk_sample.isnull().sum(axis = 0))
print("Total rows: ")
print(risk_sample.shape[0])

PunktID                 0
E                       0
N                       0
Kommun                  0
Datum                   0
Temp                    0
Tmedel                  0
Nederbord               0
RH                   3989
Vindhastighet           0
Vindriktning     11068726
FFMC                    0
DMC                     0
DC                      0
ISI                     0
BUI                     0
FWI                     0
FWI_index               0
HBV_o               39890
HBV_u               39890
HBV                 39890
HBV_index           39890
Gras              6777056
dtype: int64
Total rows: 
17474853


In [12]:
fireRisk = risk_sample.drop(columns=['Nederbord',
       'RH', 'Vindhastighet', 'Vindriktning', 'FFMC', 'DMC', 'DC', 'ISI',
       'BUI', 'HBV_o', 'HBV_u', 'HBV', 'HBV_index',
       'Gras'])
fireRisk

,PunktID,E,N,Kommun,Datum,Temp,Tmedel,FWI,FWI_index
0,1,454334,6690301,2021.0,2000-04-01,"1,8","1,6",0,1.0
1,1,454334,6690301,2021.0,2000-04-02,"-2,3","-2,3",0,1.0
2,1,454334,6690301,2021.0,2000-04-03,"-1,4",-3,0,1.0
3,1,454334,6690301,2021.0,2000-04-04,"0,1","-0,2",0,1.0
4,1,454334,6690301,2021.0,2000-04-05,"0,4","-2,8",0,1.0
...,...,...,...,...,...,...,...,...,...
19188852,7892,375482,6733336,1737.0,2020-12-27,"0,4",0,0,-1.0
19188853,7892,375482,6733336,1737.0,2020-12-28,"-0,1",0,0,-1.0
19188854,7892,375482,6733336,1737.0,2020-12-29,"0,3",0,0,-1.0
19188855,7892,375482,6733336,1737.0,2020-12-30,"-0,1",0,0,-1.0


In [16]:
#Values needs to be float to be able to do means, to convert to float we first change all ',' to '.'.
fireRisk['FWI'] = fireRisk['FWI'].str.replace(',','.')
fireRisk['Temp'] = fireRisk['Temp'].str.replace(',','.')
fireRisk['Tmedel'] = fireRisk['Tmedel'].str.replace(',','.')
fireRisk['FWI'] = fireRisk['FWI'].astype(float)
fireRisk['Temp'] = fireRisk['Temp'].astype(float)
fireRisk['Tmedel'] = fireRisk['Tmedel'].astype(float)
fireRisk['Kommun'] = fireRisk['Kommun'].astype(int)
#Datum needs to be in datetime format, here we use pd.to_datetime to convert the strings
fireRisk['Datum'] = pd.to_datetime(fireRisk['Datum'], format='%Y-%m-%d')
#For this we dont need the sweref99 coordinates and we drop these columns.
fireRiskT = fireRisk.drop(columns=['E','N'])

In [ ]:
#Grouping by Kommun and date (YYYY-MM-DD) and converting rest of values to mean values.
test = fireRiskT.groupby(['Datum','Kommun']).mean()
test = test.reset_index()
test = test.rename(columns={"Datum": "datum", "Kommun":"kommun"})

In [23]:

#Actual merge happens here
merged = pd.merge(fires_sample, test, how='inner', on=['kommun','datum'])
merged = merged.drop(columns='index')



In [ ]:
#It might be interesting to look at the days where the fires of the municipalities FWI_index is above 4
above4 = merged[merged['FWI_index']>4]
above4

In [36]:
#Filter cause to only include campfire and BBQ
above4 = filter_rows_by_values(above4, 'BEJBbrandorsakText', ['Fyrverkeri eller pyroteknik', 'Annan',
       'Barns lek med eld', 'Annan eldning', 'Avsiktlig brand',
       'Övriga gnistor',
       'Återantändning av brand från tidigare räddningsinsats',
       'Självantändning - biologisk eller kemisk', 'Blixtnedslag', 'Heta arbeten',
       'Rökning', 'Tågbromsning', 'Fel i utrustning', 'Uppgift saknas'])
above4

,ar,datum,tid,kommun,kommunKortNamn,verksamhetText,sweref99Norr,sweref99Ost,BEJBbrandorsakText,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2,Temp,Tmedel,FWI,FWI_index
1420,2000,2000-05-07,13:16:00,380,Uppsala,Verksamhet inte knuten till en byggnad,6635996.0,647573.0,Okänd,0,0,100,18.200000,10.047368,21.178947,4.368421
1532,2000,2000-05-09,01:30:00,1980,Västerås,Verksamhet inte knuten till en byggnad,NaN,NaN,Okänd,0,0,100,22.366667,15.066667,20.300000,4.111111
1533,2000,2000-05-09,09:09:00,1980,Västerås,Verksamhet inte knuten till en byggnad,6607977.0,582930.0,Okänd,0,0,1,22.366667,15.066667,20.300000,4.111111
1534,2000,2000-05-09,21:47:00,1980,Västerås,Verksamhet inte knuten till en byggnad,6609205.0,585442.0,Okänd,0,0,1,22.366667,15.066667,20.300000,4.111111
1539,2000,2000-05-09,08:03:00,180,Stockholm,Verksamhet inte knuten till en byggnad,NaN,NaN,Okänd,0,8,0,22.900000,14.650000,22.450000,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85641,2020,2020-08-27,16:35:00,1983,Köping,Verksamhet inte knuten till en byggnad,6593620.0,563258.0,Okänd,70,0,0,19.350000,12.833333,26.483333,5.166667
85644,2020,2020-08-27,16:53:00,140,Nykvarn,Verksamhet inte knuten till en byggnad,6557858.0,642314.0,Okänd,0,30,0,19.233333,14.333333,25.333333,5.000000
85653,2020,2020-08-27,22:21:00,180,Stockholm,Verksamhet inte knuten till en byggnad,6572544.0,673272.0,Okänd,0,10,0,19.550000,16.000000,27.550000,5.500000
85667,2020,2020-08-28,14:57:00,1980,Västerås,Verksamhet inte knuten till en byggnad,6609656.0,589536.0,Okänd,0,0,3,17.922222,11.666667,24.066667,4.777778
